In [1]:
# from pyspark.sql import functions as F
# from pyspark.ml.feature import RegexTokenizer, StopWordsRemover

# # =====================================================
# # STEP 1: Clean the 'resp.text' field (Business Responses)
# # =====================================================
# df_review = df_review.withColumn("resp_text", F.col("resp.text"))
# # remove (Translated by Google)
# df_review = df_review.withColumn("resp_text", F.regexp_replace("resp_text", r'\(Translated by Google\)', ''))
# # remove (Original) and everything after
# df_review = df_review.withColumn("resp_text", F.regexp_replace("resp_text", r'\(Original\).*', ''))
# # replace newlines with spaces
# df_review = df_review.withColumn("resp_text", F.regexp_replace("resp_text", r'[\r\n]+', ' '))
# # trim extra spaces
# df_review = df_review.withColumn("resp_text", F.trim("resp_text"))

# # =====================================================
# # STEP 2: Clean the 'text' column (Customer Reviews)
# # =====================================================
# df_review = df_review.withColumn("review_text", F.col("text"))
# # remove (Translated by Google) if present
# df_review = df_review.withColumn("review_text", F.regexp_replace("review_text", r'\(Translated by Google\)', ''))
# # remove (Original) and everything after
# df_review = df_review.withColumn("review_text", F.regexp_replace("review_text", r'\(Original\).*', ''))
# # replace newlines with spaces
# df_review = df_review.withColumn("review_text", F.regexp_replace("review_text", r'[\r\n]+', ' '))
# # trim extra spaces
# df_review = df_review.withColumn("review_text", F.trim("review_text"))

# # =====================================================
# # STEP 3: Tokenization for Customer Reviews
# # =====================================================
# review_tokenizer = RegexTokenizer(inputCol="review_text", outputCol="review_tokens", pattern="\\W")
# df_tokens = review_tokenizer.transform(df_review)

# # =====================================================
# # STEP 4: Tokenization for Business Responses
# # =====================================================
# resp_tokenizer = RegexTokenizer(inputCol="resp_text", outputCol="resp_tokens", pattern="\\W")
# df_tokens = resp_tokenizer.transform(df_tokens)

# # =====================================================
# # STEP 5: Stopword Removal for Customer Reviews
# # =====================================================
# review_remover = StopWordsRemover(inputCol="review_tokens", outputCol="review_filtered_tokens")
# df_clean = review_remover.transform(df_tokens)

# # =====================================================
# # STEP 6: Stopword Removal for Business Responses
# # =====================================================
# resp_remover = StopWordsRemover(inputCol="resp_tokens", outputCol="resp_filtered_tokens")
# df_clean = resp_remover.transform(df_clean)

# # =====================================================
# # STEP 7: TF-IDF for Customer Reviews
# # =====================================================
# from pyspark.ml.feature import HashingTF, IDF

# # HashingTF for customer reviews
# review_hashingTF = HashingTF(inputCol="review_filtered_tokens", outputCol="review_rawFeatures")
# review_featurized = review_hashingTF.transform(df_clean)

# # IDF for customer reviews
# review_idf = IDF(inputCol="review_rawFeatures", outputCol="review_features")
# review_idfModel = review_idf.fit(review_featurized)
# df_with_review_features = review_idfModel.transform(review_featurized)

# # =====================================================
# # STEP 8: TF-IDF for Business Responses
# # =====================================================
# # HashingTF for business responses
# resp_hashingTF = HashingTF(inputCol="resp_filtered_tokens", outputCol="resp_rawFeatures")
# resp_featurized = resp_hashingTF.transform(df_with_review_features)

# # IDF for business responses
# resp_idf = IDF(inputCol="resp_rawFeatures", outputCol="resp_features")
# resp_idfModel = resp_idf.fit(resp_featurized)
# df_final = resp_idfModel.transform(resp_featurized)


# Official

In [2]:
from pyspark.sql import SparkSession
import numpy as np
import os
from pyspark.sql import functions as F
from pyspark.sql.functions import col, concat_ws, rand

In [3]:
spark = SparkSession.builder \
        .appName("Hackathon") \
        .master("local[*]") \
        .getOrCreate()

pathing_review = "datasets/review_data/"
arr = np.array(os.listdir(pathing_review))
reviewData_files = pathing_review + arr

pathing_metadata = "datasets/review_metadata/"
arr = np.array(os.listdir(pathing_metadata))
reviewMetadata_files = pathing_metadata + arr

df_review = spark.read.json(list(reviewData_files)).dropna(subset="text").drop_duplicates()
df_metadata = spark.read.json(list(reviewMetadata_files)).dropna(subset="category").drop_duplicates().select(["gmap_id", "category"])

df_joined = df_review.join(df_metadata, on="gmap_id", how="inner").withColumn("category_str", concat_ws(", ", col("category"))).withColumn("random_order", rand()).orderBy("random_order").drop("random_order").limit(12000)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/28 00:31:18 WARN Utils: Your hostname, Asyrafs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.18.78 instead (on interface en0)
25/08/28 00:31:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/28 00:31:18 WARN Utils: Your hostname, Asyrafs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.18.78 instead (on interface en0)
25/08/28 00:31:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust

In [4]:
df_joined.printSchema()

root
 |-- gmap_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pics: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- url: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |-- rating: long (nullable = true)
 |-- resp: struct (nullable = true)
 |    |-- text: string (nullable = true)
 |    |-- time: long (nullable = true)
 |-- text: string (nullable = true)
 |-- time: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- category: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- category_str: string (nullable = false)



In [ ]:
system_prompt = """
You must classify each review into exactly ONE of these labels. Respond with ONLY the label text, nothing else.

Valid labels (respond with exactly this text):
- Advertisement
- Irrelevant content
- Rant without visit
- Low quality review
- Acceptable review

Classification rules:
1. Advertisement: Contains promotional content, links, or sales pitches
2. Irrelevant content: Not about the business location (personal stories, unrelated topics)
3. Rant without visit: Complaints from someone who clearly hasn't visited
4. Low quality review: Very short, just emojis, or provides no useful information
5. Acceptable review: Genuine experience at the location with useful details

Business category: %s

IMPORTANT: Respond with ONLY one of the five labels above. Do not include numbers, prefixes, or explanations.
"""

In [6]:
df_batch = df_joined.withColumn(
    "json_request",
    F.to_json(F.struct(
        F.concat(F.lit("request-"), F.monotonically_increasing_id()).alias("custom_id"),
        F.lit("POST").alias("method"),
        F.lit("/v1/chat/completions").alias("url"),
        F.struct(
            F.lit("gpt-4o-mini").alias("model"),  # Use a more standard model
            F.array(
                F.struct(
                    F.lit("system").alias("role"),
                    F.format_string(system_prompt, F.col("category_str")).alias("content")
                ),
                F.struct(
                    F.lit("user").alias("role"),
                    F.concat(F.lit("Review: "), F.col("text")).alias("content")
                )
            ).alias("messages"),
            F.lit(150).alias("max_completion_tokens")  # Increased token limit
        ).alias("body")
    ))
)

In [7]:
df_batch.select("json_request").coalesce(1).write.mode("overwrite").json("output/batchinput_temp")


In [8]:
spark.stop()

In [9]:
import glob
import shutil

# Find the part file and rename to batchinput.jsonl
part_file = glob.glob("output/batchinput_temp/part-*.json")[0]
shutil.move(part_file, "batchinput.jsonl")

'batchinput.jsonl'

In [10]:
import json

# Fix double-encoded batchinput.jsonl to valid JSONL for OpenAI Batch API
with open("batchinput.jsonl", "r") as infile, open("batchinput_fixed.jsonl", "w") as outfile:
    for line in infile:
        # Each line is a JSON object with a string value under 'json_request'
        obj = json.loads(line)
        # Parse the string value to a dict
        fixed = json.loads(obj["json_request"])
        # Write as a single JSON object per line
        outfile.write(json.dumps(fixed) + "\n")

print("Wrote fixed file: batchinput_fixed.jsonl")

Wrote fixed file: batchinput_fixed.jsonl


In [11]:
import os

def split_jsonl(input_path, lines_per_file=1000):
    os.makedirs('batches', exist_ok=True)
    with open(input_path, 'r') as infile:
        file_count = 0
        lines = []
        for i, line in enumerate(infile, 1):
            lines.append(line)
            if i % lines_per_file == 0:
                with open(f'batches/batch_{file_count}.jsonl', 'w') as out:
                    out.writelines(lines)
                lines = []
                file_count += 1
        if lines:
            with open(f'batches/batch_{file_count}.jsonl', 'w') as out:
                out.writelines(lines)
    print(f"Split into {file_count+1} files in 'batches/' directory.")

split_jsonl('batchinput_fixed.jsonl', lines_per_file=1000)

Split into 13 files in 'batches/' directory.


In [ ]:
import glob
from openai import OpenAI
import time
import os
client = OpenAI()
batch_files = sorted(glob.glob("batches/batch_*.jsonl"))
os.makedirs("batch_results", exist_ok=True)
for batch_path in batch_files:
    # 1. Upload batch file
    upload = client.files.create(file=open(batch_path, "rb"), purpose="batch")
    print(f"Uploaded {batch_path} as file ID: {upload.id}")

    # 2. Submit batch job
    job = client.batches.create(
        input_file_id=upload.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={"description": f"Batch job for {batch_path}"}
    )
    print(f"Submitted batch job ID: {job.id}")

    # 3. Poll for job completion
    while True:
        status = client.batches.retrieve(job.id)
        print(f"Job {job.id} status: {status.status}")
        if status.status in ["completed", "failed", "cancelled"]:
            break
        time.sleep(10)  # Wait before checking again

    # 4. Download results if completed
    if status.status == "completed" and status.output_file_id:
        print(f"Batch {batch_path} completed. Downloading results...")
        result_file = client.files.retrieve(status.output_file_id)
        content = client.files.content(status.output_file_id)
        out_path = os.path.join("batch_results", f"results_{os.path.basename(batch_path)}")
        with open(out_path, "wb") as f:
            f.write(content.read())
        print(f"Results saved to {out_path}")
    else:
        print(f"Batch {batch_path} did not complete successfully. Check errors.")

    # Optional: Pause between batches to avoid rate limits
    time.sleep(10)

In [25]:
# SIMPLIFIED VERSION - Just run this directly
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, concat_ws, rand, row_number, lit
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import json
import glob
import numpy as np
import os

# Stop existing session and create fresh one
try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("ReviewClassificationSimple") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "false") \
    .getOrCreate()

print("Fresh Spark session created")

# Recreate original data
print("Recreating original dataset...")
pathing_review = "datasets/review_data/"
arr = np.array(os.listdir(pathing_review))
reviewData_files = pathing_review + arr

pathing_metadata = "datasets/review_metadata/"
arr = np.array(os.listdir(pathing_metadata))
reviewMetadata_files = pathing_metadata + arr

df_review = spark.read.json(list(reviewData_files)).dropna(subset="text").drop_duplicates()
df_metadata = spark.read.json(list(reviewMetadata_files)).dropna(subset="category").drop_duplicates().select(["gmap_id", "category"])

df_joined = df_review.join(df_metadata, on="gmap_id", how="inner") \
    .withColumn("category_str", concat_ws(", ", col("category"))) \
    .withColumn("random_order", rand()) \
    .orderBy("random_order") \
    .drop("random_order") \
    .limit(12000)

# Add row numbers
window = Window.orderBy(lit(1))
df_with_row_num = df_joined.withColumn("row_number", row_number().over(window) - 1)

print(f"Created DataFrame with {df_with_row_num.count()} rows")

# Process batch results
print("Processing batch results...")
result_files = glob.glob("batch_results/results_batch_*.jsonl")
all_results = []

for file_path in result_files:
    with open(file_path, 'r') as f:
        for line in f:
            result = json.loads(line)
            custom_id = result['custom_id']
            row_num = int(custom_id.split('-')[1])
            
            if result.get('error') is None and result.get('response', {}).get('status_code') == 200:
                classification = result['response']['body']['choices'][0]['message']['content'].strip()
                all_results.append({
                    'row_number': row_num,
                    'classification': classification
                })
            else:
                all_results.append({
                    'row_number': row_num,
                    'classification': 'Error'
                })

all_results.sort(key=lambda x: x['row_number'])
print(f"Processed {len(all_results)} results")

# Create results DataFrame
results_schema = StructType([
    StructField("row_number", IntegerType(), True),
    StructField("classification", StringType(), True)
])

df_results = spark.createDataFrame(all_results, schema=results_schema)

# Perform join
print("Performing join...")
df_final = df_with_row_num.join(df_results, on="row_number", how="left").drop("row_number")

# Test basic operations
print("Testing DataFrame...")
try:
    total_count = df_final.count()
    print(f"Total rows: {total_count}")
    
    print("\nClassification distribution:")
    classification_results = df_final.groupBy("classification").count().collect()
    
    # Sort and display results manually to avoid show() issues
    sorted_results = sorted(classification_results, key=lambda x: x['count'], reverse=True)
    for row in sorted_results:
        print(f"{row['classification']}: {row['count']}")
    
except Exception as e:
    print(f"Error during operations: {e}")
    # Let's try a simpler approach - just save the data
    print("Attempting direct save...")

# Save results
print("\nSaving results...")
try:
    # Try saving with fewer partitions
    df_final.coalesce(1).write.mode("overwrite").parquet("output/classified_reviews")
    print("Parquet save successful!")
    
    # Also save a sample as CSV for easy viewing
    df_final.select("text", "category_str", "rating", "classification") \
           .limit(1000) \
           .coalesce(1) \
           .write.mode("overwrite") \
           .option("header", "true") \
           .csv("output/sample_classified_reviews")
    print("Sample CSV saved!")
    
except Exception as e:
    print(f"Save error: {e}")
    # Try JSON format instead
    try:
        df_final.coalesce(1).write.mode("overwrite").json("output/classified_reviews_json")
        print("JSON save successful!")
    except Exception as e2:
        print(f"JSON save also failed: {e2}")

print("Process completed!")

# Optional: Show a few sample rows if possible
try:
    print("\nSample results:")
    sample_data = df_final.select("text", "category_str", "classification").limit(5).collect()
    for i, row in enumerate(sample_data):
        print(f"{i+1}. {row['classification']} | {row['category_str']} | {row['text'][:100]}...")
except:
    print("Could not display sample results, but data should be saved")

Fresh Spark session created
Recreating original dataset...


25/08/28 05:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/28 05:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Created DataFrame with 12000 rows
Processing batch results...
Processed 12000 results
Performing join...
Testing DataFrame...


25/08/28 05:51:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/28 05:51:53 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Total rows: 12000

Classification distribution:


...

25/08/28 05:53:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/28 05:53:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Parquet save successful!


25/08/28 05:53:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/08/28 05:53:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


Sample CSV saved!
Process completed!

Sample results:


1. Acceptable review | Fabric store, Baking supply store, Bead store, Craft store, Knit shop, Picture frame shop, Scrapbooking store, Seasonal goods store, Sewing shop, Yarn store | Had what I needed....
2. Acceptable review | American restaurant, Breakfast restaurant, Family restaurant, Hamburger restaurant | Good food n really good service...
3. Acceptable review | Fast food restaurant, Breakfast restaurant, Chicken restaurant, Hamburger restaurant | The breakfast sandwich was cold!...
4. Label: Low quality review | Mediterranean restaurant, Chicken shop, Coffee shop, Health food restaurant, Juice shop, Salad shop | They really care here you can tell as soon as you walk in. Foods wonderful give it a try!...
5. Acceptable review | Hamburger restaurant, American restaurant, Fast food restaurant, Hot dog restaurant, Hot dog stand, Restaurant | Got to love Five Guys Burgers and Fries worth the wait every time...


In [34]:
spark.stop()

In [27]:
import pandas as pd
df_classified_reviews = pd.read_parquet("output/classified_reviews")

In [29]:
df_classified_reviews[["text", "category_str", "rating", "classification"]].head()

,text,category_str,rating,classification
0,Great people. A one stop store for anything yo...,"Grocery store, Propane supplier",5,Acceptable review
1,Always spend too much!,"Warehouse store, Department store",5,Acceptable review
2,Bring your appetite. Best tortas in town.,"Mexican restaurant, Breakfast restaurant, Burr...",5,Low quality review
3,I like shopping here,"Sporting goods store, Boat dealer, Camping sto...",5,Acceptable review
4,Fine if you want to browse for video games. Th...,"Video game store, Cell phone store, Collectibl...",3,Acceptable review


In [ ]:
import pandas as pd
import re

def clean_classification_labels(df, classification_col='classification'):
    """
    Clean inconsistent classification labels to standard format
    """
    # Define the target labels
    target_labels = [
        'Advertisement',
        'Irrelevant content', 
        'Rant without visit',
        'Low quality review',
        'Acceptable review'
    ]
    
    def standardize_label(label):
        if pd.isna(label) or label == 'Error':
            return 'Error'
        
        # Convert to string and clean
        label = str(label).strip()
        
        # Remove common prefixes
        prefixes_to_remove = [
            r'^Label:\s*',
            r'^Classification:\s*',
            r'^Classified as:\s*',
            r'^Level:\s*',
            r'^\d+\.\s*',  # Remove numbers like "5. "
            r'^Classify.*?labels?:\s*',
        ]
        
        for prefix in prefixes_to_remove:
            label = re.sub(prefix, '', label, flags=re.IGNORECASE)
        
        # Handle specific variations
        label_lower = label.lower().strip()
        
        if 'advertisement' in label_lower or 'promotional' in label_lower:
            return 'Advertisement'
        elif 'irrelevant' in label_lower:
            return 'Irrelevant content'
        elif 'rant' in label_lower and 'visit' in label_lower:
            return 'Rant without visit'
        elif 'low quality' in label_lower:
            return 'Low quality review'
        elif 'acceptable' in label_lower or 'acceptance' in label_lower:
            return 'Acceptable review'
        else:
            # Try exact match after cleaning
            for target in target_labels:
                if target.lower() == label_lower:
                    return target
            
            # If no match found, return original for manual inspection
            return f"UNMATCHED: {label}"
    
    # Apply cleaning
    df_clean = df.copy()
    df_clean[f'{classification_col}_cleaned'] = df_clean[classification_col].apply(standardize_label)
    
    return df_clean


# Clean the labels
df_cleaned = clean_classification_labels(df_classified_reviews)

# Show before and after comparison
print("BEFORE CLEANING:")
print(df_classified_reviews['classification'].value_counts())

print("\nAFTER CLEANING:")
print(df_cleaned['classification_cleaned'].value_counts())

# Check unmatched labels
unmatched = df_cleaned[df_cleaned['classification_cleaned'].str.startswith('UNMATCHED')]
if len(unmatched) > 0:
    print(f"\nUNMATCHED LABELS ({len(unmatched)} rows):")
    print(unmatched['classification_cleaned'].value_counts())

# Replace original column with cleaned version
df_classified_reviews['classification'] = df_cleaned['classification_cleaned']

# Final clean counts
print("\nFINAL CLEANED DISTRIBUTION:")
final_counts = df_classified_reviews['classification'].value_counts()
print(final_counts)

# Calculate percentages
print("\nPERCENTAGES:")
percentages = (final_counts / len(df_classified_reviews) * 100).round(1)
for label, count in final_counts.items():
    pct = percentages[label]
    print(f"{label}: {count} ({pct}%)")

# Save cleaned dataset
df_classified_reviews.to_csv('output/classified_reviews_cleaned.csv', index=False)
print(f"\nCleaned dataset saved to 'output/classified_reviews_cleaned.csv'")
print(f"Total rows: {len(df_classified_reviews)}")

In [33]:
df_classified_reviews['classification'].value_counts()

classification
Acceptable review     8419
Low quality review    2536
Rant without visit     867
Irrelevant content     151
Advertisement           27
Name: count, dtype: int64

In [13]:
# # Test a single request to OpenAI API with 'gpt-5-nano' to verify model access and request validity
# from openai import OpenAI
# client = OpenAI()
# response = client.chat.completions.create(
#     model="gpt-5-nano",
#     messages=[
#         {
#             "role": "system",
#             "content": (
#                 "Classify the following review into one of these labels: "
#                 "1. Advertisement (promotional content or links) "
#                 "2. Irrelevant content (not about the location) "
#                 "3. Rant without visit (complaints from non-visitors) "
#                 "4. Low quality review (little info or random emojis) "
#                 "5. Acceptable review (genuine experience at the location)\n\n"
#                 "Examples:\n"
#                 "\"Best pizza! Visit www.pizzapromo.com for discounts!\" → Advertisement\n"
#                 "\"I love my new phone, but this place is too noisy.\" → Irrelevant content\n"
#                 "\"Never been here, but I heard it is terrible.\" → Rant without visit\n"
#                 "\"Good\" → Low quality review\n"
#                 "\"I have gotten two piercings done by Adner. He is very clean, calm, and knowledgeable...\" → Acceptable review\n\n"
#                 "Helpful tags to help decide what classification types to assign: Dermatologist, Cancer treatment center, Medical Center, Skin care clinic\n\n"
#             )
#         },
#         {
#             "role": "user",
#             "content": "Review: As always, it was easy to make an appointment and I was seen on time . The staff was great and made me feel like a valued customer! I highly recommend this Office"
#         }
#     ],
#     max_completion_tokens=1024
# )
# print(response)